In [1]:

import os

import torch
import pickle
import subprocess

import timeit
import select
import json
from Lean4Gym import Lean4Gym, ProofState
# print(torch.__version__)
from torch.nn.parallel import DataParallel  
# torch.cuda.set_device(2)

from model import policy_model
from model import value_model
from trainer import Trainer
from mcts import State
from mcts import Node
from mcts import MCTS

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

/home2/wanglei/anaconda3/envs/wanglei/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home2/wanglei/anaconda3/envs/wanglei/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-07-04 15:40:29,221	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-04 15:40:29 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/home2/wanglei/Project/model/pythia2.8b_choose', tokenizer='/home2/wanglei/Project/model/pythia2.8b_choose', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-04 15:40:32 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 07-04 15:40:32 selector.py:25] Using XFormers backend.
INFO 07-04 15:40:36 model_runner.py:104] Loading model weights took 5.1857 GB
INFO 07-04 15:40:36 gpu_executor.py:94] # GPU blocks: 552, # CPU blocks: 819
INFO 07-04 15:40:38 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-04 15:40:38 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-04 15:40:42 model_runner.py:867] Graph capturing finished in 4 secs.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model_name_or_path = "/home2/wanglei/Project/model/pythia2.8b_choose"
tokenizer = transformers.GPTNeoXTokenizerFast.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


参数设置

In [3]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu') 

args = {
    
    'batch_size': 4,
    'numIters': 10,                                # Total number of training iterations
    'num_simulations': 100,                         # Total number of MCTS simulations to run when deciding on a move to play
    'numEps': 20,                                  # Number of full games (episodes) to run during each iteration
    'numItersForTrainExamplesHistory': 20,
    'epochs': 10,                                    # Number of epochs of training per iteration
    'checkpoint_path': 'latest.pth',                 # location to save latest set of weights
    'TACRIC_NUMBER': 8,
    'feature_size':100
    # 'MAX_ROUND_NUMBER' : 10
}

feature_size = args['feature_size']  # 特征向量的size
time_out = 600

导入训练好的策略网络和价值网络

In [4]:
policyModel = policy_model(feature_size*2, device).to(device)
valueModel = value_model(feature_size, device).to(device)

checkpoint_policy = torch.load("/home2/wanglei/Project/alphazero_leanrepl_gpu/policy_model")
policyModel.load_state_dict(checkpoint_policy['state_dict'])

checkpoint_value = torch.load("//home2/wanglei/Project/alphazero_leanrepl_gpu/value_model")
valueModel.load_state_dict(checkpoint_value['state_dict'])

<All keys matched successfully>

导入待证明定理：$ 𝐶_n^{n−k}=𝐶_n^k   (k ≤ n) $ ,获取该定理状态并设置为根节点:

In [5]:

lean_workdir = "/home2/wanglei/Project" # Lean工程的根目录
lean_file = "demo.lean"   # 待证明定理的Lean文件
lean = Lean4Gym(lean_workdir, lean_file)
state = lean.getInitState()
init_state = State(state)
node = Node()
node.set_state(init_state)


In [6]:
start = timeit.default_timer()

print("开始搜索策略")
mcts = MCTS(node, policyModel, valueModel, args, device)
timespend = mcts.runmcts(lean, time_out)

end = timeit.default_timer()

print ("所用时间：{}".format(end-start))

开始搜索策略


Node: 8480346000156, Q/N: 0.0/0, state: <mcts.State object at 0x7b67c02cbb20>
成功策略：
成功路径策略：
rw [choose_eq_factorial_div_factorial hk]
rw [choose_eq_factorial_div_factorial]
rw [Nat.sub_sub_self hk]
rw [mul_comm]
exact Nat.sub_le _ _
成功路径状态：
n k : ℕ
hk : k ≤ n
⊢ Nat.choose n (n - k) = Nat.choose n k
n k : ℕ
hk : k ≤ n
⊢ Nat.choose n (n - k) = n ! / (k ! * (n - k)!)
n k : ℕ
hk : k ≤ n
⊢ n ! / ((n - k)! * (n - (n - k))!) = n ! / (k ! * (n - k)!)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n
n k : ℕ
hk : k ≤ n
⊢ n ! / ((n - k)! * k !) = n ! / (k ! * (n - k)!)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n
n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n
no goals
所用时间：True


In [7]:
state1 = lean.run_tactic(state, [" rw[choose_eq_factorial_div_factorial hk]"])
print(state1.tacticState)

n k : ℕ
hk : k ≤ n
⊢ Nat.choose n (n - k) = n ! / (k ! * (n - k)!)


In [8]:
state2 = lean.run_tactic(state1, ["rw[choose_eq_factorial_div_factorial]"])
print(state2.tacticState)

n k : ℕ
hk : k ≤ n
⊢ n ! / ((n - k)! * (n - (n - k))!) = n ! / (k ! * (n - k)!)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n


In [9]:
state3 = lean.run_tactic(state2, ["rw[Nat.sub_sub_self hk]"])
print(state3.tacticState)

n k : ℕ
hk : k ≤ n
⊢ n ! / ((n - k)! * k !) = n ! / (k ! * (n - k)!)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n


In [10]:
state4 = lean.run_tactic(state3, ["rw [mul_comm]"])
print(state4.tacticState)

n k : ℕ
hk : k ≤ n
⊢ n - k ≤ n


In [11]:
state5 = lean.run_tactic(state4, ["exact Nat.sub_le _ _"])
print(state5.tacticState)

no goals
